# IMPORT DATA, PACKAGES, AND FUNCTIONS

## import the relevant functions and packages

In [1]:
import pandas as pd
import numpy as np
from test_case_generator_functions import alphabet

# TEST CASE TYPE REFERENCE INFORMATION

In [2]:
uid = 'UID-256' 
theme = 'Names where name parts are Modified'
category = 'Transpositions'
sub_category = '2 transpositions (different name part)'
entity_type = 'Individual'

## download the OFAC list from the web

In [3]:
ofac_list_download = pd.read_csv('https://www.treasury.gov/ofac/downloads/sdn.csv', header=None)

In [4]:
ofac_list = ofac_list_download[[0,1,2]]
ofac_list.columns = ['uid', 'name', 'entity_type']

# FILTER FOR THE REQUIREMENTS OF THE TEST CASE TYPE

## filter for the requirements of the specific test case type

In [5]:
ofac_list_filtered = ofac_list[(ofac_list.entity_type == 'individual')] # only evaluate individuals

## randomly choose 10 rows

In [6]:
while True:
    ofac_list_sampled = ofac_list_filtered.sample(n = 10)
    cnt = [0] * 10
    for ind, name in enumerate(ofac_list_sampled['name']):
        split_name = name.upper().split(' ')
        
        for i in split_name:
            if all(j not in alphabet() for j in i):
                split_name.remove(i) # remove word with all numbers
            elif len(i) == 1:
                split_name.remove(i) # remove word with only 1 letter
                
        cnt[ind] += len(split_name)

    if all(i >= 2 for i in cnt):
        break

ofac_list_sampled

,uid,name,entity_type
9496,35479,"SHCHEGLOV, Nikolay Mikhaylovich",individual
8855,33513,"BASKAU, Dzmitriy Yurievich",individual
4244,18988,"KHURI, Mudalal",individual
1392,10265,"CASTRO GARZON, Victor Hugo",individual
9518,35501,"FADINA, Oksana Nikolayevna",individual
10658,37866,"ARTYAKOV, Vladimir Vladimirovich",individual
3483,16536,"KOSARAYANIFARD, Pejman Mahmood",individual
9756,35766,"NAVKA, Tatiana Aleksandrovna",individual
9675,35658,"KUZMIN, Mikhail Vladimirovich",individual
2041,11807,"SANCHEZ SILVA, Elkin Alexis",individual


# CREATE THE TEST CASES

## create blank final test cases table

In [7]:
final_test_cases = pd.DataFrame(columns=['UID', 'Theme','Category','Sub-category','Entity-Type','Test Case ID' , 'OFAC List UID', 'Original Name','Test Case Name'])
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name


## run loop to generate the test cases

In [8]:
for index, row in ofac_list_sampled.iterrows():
    original_name = row['name'].upper()
    split_original_name = original_name.split(' ') # split name into word
    final_test_name = split_original_name.copy()
    
    for i in split_original_name:
        if all(j not in alphabet() for j in i):
            split_original_name.remove(i) # remove word with all numbers
        elif len(i) == 1:
            split_original_name.remove(i) # remove word with only 1 letter
        
    for i in range(2):
        replace_word = np.random.choice(split_original_name) # randomly choose word to be transposed
        split_original_name.remove(replace_word) # remove chosen word
        
        random_list = list(range(len(replace_word))) # randomly choose letter to be transposed
        while True:
            random_letter_index = np.random.choice(random_list)
            random_letter = replace_word[random_letter_index]
            if random_letter in alphabet():
                break
        random_list.remove(random_letter_index) # remove chosen letter index
        
        random_place = np.random.choice(random_list) # randomly choose place to transpose to
        
        replace_word_list = list(replace_word)
        del replace_word_list[random_letter_index] # remove chosen letter
        replace_word_list.insert(random_place, random_letter) # add transposed letter
        
        final_test_name[final_test_name.index(replace_word)] = ''.join(replace_word_list)

    final_test_name = ' '.join(final_test_name)
    
    final_test_cases.loc[len(final_test_cases)] = [uid, theme, category, sub_category, entity_type, uid + ' - ' + str(index), row['uid'], row['name'], final_test_name] # append to the dataframe

final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name
0,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 9496,35479,"SHCHEGLOV, Nikolay Mikhaylovich","HSCHEGLOV, NIKOLAY MAIKHYLOVICH"
1,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 8855,33513,"BASKAU, Dzmitriy Yurievich","BAASKU, DZMITRIY YURIEICHV"
2,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 4244,18988,"KHURI, Mudalal","HKURI, UDALALM"
3,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 1392,10265,"CASTRO GARZON, Victor Hugo","CASTRO NGARZO, VICTOR HGUO"
4,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 9518,35501,"FADINA, Oksana Nikolayevna","FADIA,N OKSANA NIKOLAEVNYA"
5,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 10658,37866,"ARTYAKOV, Vladimir Vladimirovich","ARTYAKOV, VLADIRMI VLADIIROVICMH"
6,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 3483,16536,"KOSARAYANIFARD, Pejman Mahmood","KOSARAYANIFARD, JPEMAN HMAMOOD"
7,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 9756,35766,"NAVKA, Tatiana Aleksandrovna","KNAVA, TNATIAA ALEKSANDROVNA"
8,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 9675,35658,"KUZMIN, Mikhail Vladimirovich","KUZMIN, MIHAIKL VLIADIMROVICH"
9,UID-256,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Individual,UID-256 - 2041,11807,"SANCHEZ SILVA, Elkin Alexis","SANCHEZ SIVAL, ELNKI ALEXIS"
